---
# Image Matching Challenge 2022
##### Register two images from different viewpoints
---

### **Objective**
The [Image Matching Challenge 2022](https://www.kaggle.com/competitions/image-matching-challenge-2022/overview) Register two images from different viewpoints Kaggle competition is focused on developing computer vision models that can match pairs of images that show the same scene or object from different viewpoints. Specifically, the competition requires participants to develop algorithms that can register two images by estimating the 3D transformation that aligns them. The competition provides a dataset of image pairs along with ground-truth correspondences between the images, and participants are evaluated based on the accuracy of their registration algorithm. The goal of the competition is to promote research and development in the area of image registration, which has important applications in fields such as robotics, augmented reality, and remote sensing.


In this notebook, I provide an overview of the competition and comprehensive solution to the Image Matching Challenge 2022 Kaggle competition, providing an end-to-end implementation for the problem statement, from data exploration and preprocessing to model training and submission.


### **Data**

[The Dataset](https://www.kaggle.com/competitions/image-matching-challenge-2022/data) for the Image Matching Challenge 2022 consists of two parts: the training dataset and the test dataset.

- The training dataset contains 5,000 pairs of images, where each pair of images shows the same object from two different viewpoints. The images are stored as grayscale images in JPEG format and have a resolution of 240 x 320 pixels. Each pair of images is stored in a separate directory, with the two images named as "left.jpg" and "right.jpg".

- The test dataset contains 10,000 pairs of images, with the same format as the training dataset. However, the test dataset does not have corresponding ground truth labels, which makes it challenging to evaluate the performance of different image matching algorithms.

Both the training and test datasets are provided as compressed ZIP files that can be downloaded from the competition page on Kaggle.

**Note:** This notebook has more than 25mb, So I can only upload code.

---
### Table of Contents

- Imports       
1. Background Information
2. Setup
3. Helper Functions
4. Dataset Exploration and Preprocessing
5. Baseline
6. Submission
---





## Imports
---

In [ ]:
print("\n... IMPORTS STARTING ...\n")

print("\n\tVERSION INFORMATION")

# Machine Learning and Data Science Imports
import tensorflow as tf; print(f"\t\t– TENSORFLOW VERSION: {tf.__version__}");
import tensorflow_hub as tfhub; print(f"\t\t– TENSORFLOW HUB VERSION: {tfhub.__version__}");
import tensorflow_addons as tfa; print(f"\t\t– TENSORFLOW ADDONS VERSION: {tfa.__version__}");
import pandas as pd; pd.options.mode.chained_assignment = None;
import numpy as np; print(f"\t\t– NUMPY VERSION: {np.__version__}");
import sklearn; print(f"\t\t– SKLEARN VERSION: {sklearn.__version__}");
from sklearn.preprocessing import RobustScaler, PolynomialFeatures
from pandarallel import pandarallel; pandarallel.initialize();
from sklearn.model_selection import GroupKFold, StratifiedKFold
from scipy.spatial import cKDTree

# RAPIDS
# import cudf, cupy, cuml
# from cuml.neighbors import NearestNeighbors
# from cuml.manifold import TSNE, UMAP

# Built In Imports
from kaggle_datasets import KaggleDatasets
from collections import Counter
from datetime import datetime
from glob import glob
import warnings
import requests
import hashlib
import imageio
import IPython
import sklearn
import urllib
import zipfile
import pickle
import random
import shutil
import string
import json
import math
import time
import gzip
import ast
import sys
import io
import os
import gc
import re

# Visualization Imports
from matplotlib.colors import ListedColormap
import matplotlib.patches as patches
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm; tqdm.pandas();
import plotly.express as px
import seaborn as sns
from PIL import Image, ImageEnhance
import matplotlib; print(f"\t\t– MATPLOTLIB VERSION: {matplotlib.__version__}");
from matplotlib import animation, rc; rc('animation', html='jshtml')
import plotly
import PIL
import cv2

import plotly.io as pio
print(pio.renderers)

def seed_it_all(seed=7):
    """ Attempt to be Reproducible """
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

    
print("\n\n... IMPORTS COMPLETE ...\n")

## 1. Background Information
---

### 1.1 BASIC COMPETITION INFORMATION
---

**Participants are asked to estimate the relative pose of one image with respect to another. For each ID in the test set, you must predict the fundamental matrix between the two views.**

The classical pipeline (also implemented in examples) is to
1. Extract Local Features
2. Match Local Features in Image 1 to Local Features in Image 2
3. Filter These Feature Matches
4. Apply RANSAC (Or Some Such Similar Outlier Elimination)

You can train whole pipeline end-to-end, which is hard, or train in modular way...
1. DISK for local features
2. SuperGlue for matching
3. OANet for filtering
4. Apply some algorithmic improvements to RANSAC, to have better results.

<a href="https://www.kaggle.com/competitions/image-matching-challenge-2022/discussion/318022">[REF]</a>

**The Basic Pipeline - NOTE: This Competition is Task 1**

 <img alt="iwild" src="https://ducha-aiki.github.io/wide-baseline-stereo-blog/images/copied_from_nb/2021-05-14-IMC2020-competition-recap_files/att_00000.png" width="650px"/>

---

<br><b style="text-decoration: underline; font-family: Verdana; text-transform: uppercase;">CONTEXT</b>

For most of us, our best camera is part of the phone in our pocket. We may take a snap of a landmark, like the Trevi Fountain in Rome, and share it with friends. By itself, that photo is two-dimensional and only includes the perspective of our shooting location. Of course, a lot of people have taken photos of that fountain. Together, we may be able to create a more complete, three-dimensional view. What if machine learning could help better capture the richness of the world using the vast amounts of unstructured image collections freely available on the internet?

The process to reconstruct 3D objects and buildings from images is called Structure-from-Motion (SfM). Typically, these images are captured by skilled operators under controlled conditions, ensuring homogeneous, high-quality data. It is much more difficult to build 3D models from assorted images, given a wide variety of viewpoints, lighting and weather conditions, occlusions from people and vehicles, and even user-applied filters.

 <img alt="iwild" src="https://storage.googleapis.com/kaggle-media/competitions/google-image-matching/trevi-canvas-licensed-nonoderivs.jpg" width="650px"/>
<br>

**The first part of the problem is to identify which parts of two images capture the same physical points of a scene, such as the corners of a window. This is typically achieved with local features (key locations in an image that can be reliably identified across different views). Local features contain short description vectors that capture the appearance around the point of interest. By comparing these descriptors, likely correspondences can be established between the pixel coordinates of image locations across two or more images. This “image registration” makes it possible to recover the 3D location of the point by triangulation.**

<br>

Google employs Structure-from-Motion techniques across many Google Maps services, such as the 3D models created from StreetView and aerial imagery. In order to accelerate research into this topic, and better leverage the volume of data already publicly available, Google presents this competition in collaboration with the University of British Columbia and Czech Technical University.

<center><img src="https://storage.googleapis.com/kaggle-media/competitions/google-image-matching/image3.gif"></center>

In this code competition, you’ll create a machine learning algorithm that registers two images from different viewpoints. With access to a dataset of thousands of images to train and test your model, top-scoring notebooks will do so with the most accuracy.

If successful, you'll help solve this well-known problem in computer vision, making it possible to map the world with unstructured image collections. Your solutions will have applications in photography and cultural heritage preservation, along with Google Maps. Winners will also be invited to give a presentation as part of the Image Matching: Local Features and Beyond workshop at the Conference on Computer Vision and Pattern Recognition (CVPR) in June.

### 1.2 COMPETITION EVALUATION

---

<br><b style="text-decoration: underline; font-family: Verdana; text-transform: uppercase;">GENERAL EVALUATION INFORMATION</b>

Participants are asked to estimate the relative pose of one image with respect to another. 
* Submissions are evaluated on the **m**ean **A**verage **A**ccuracy (**mAA**) of the estimated poses. 
* Given a fundamental matrix and the hidden ground truth, we compute the error in terms of rotation and translation:
    * **rotation** --> **𝜖𝑅** (in degrees) 
    * **translation** --> **𝜖𝑇** (in meters)
* Given one threshold over each, we classify a pose as accurate if it meets both thresholds. We do this over ten pairs of thresholds, one pair at a time
    * i.e. at 1<sup>𝑜</sup> and 20 cm for the finest level
    * i.e. at 10<sup>𝑜</sup> and 5 m for the coarsest level
```python
thresholds_r = np.linspace(1, 10, 10)  # In degrees.
thresholds_t = np.geomspace(0.2, 5, 10)  # In meters.
```
* We then calculate the percentage of image pairs that meet every pair of thresholds, and average the results over all thresholds, which rewards more accurate poses. 
* As the dataset contains multiple scenes, which may have a different number of pairs, **we compute this metric separately for each scene and average it afterwards**. 
* A python implementation of this metric is available on <a href="https://www.kaggle.com/code/eduardtrulls/imc2022-tutorial-load-and-evaluate-training-data">this notebook</a>.

<br><b style="text-decoration: underline; font-family: Verdana; text-transform: uppercase;">SUBMISSION FILE</b>

For each ID in the test set, you must predict the fundamental matrix between the two views. The file should contain a header and have the following format:

```
sample_id,fundamental_matrix
a;b;c-d,0.01 0.02 0.03 0.04 0.05 0.06 0.07 0.08 0.09
a;b;e-f,0.01 0.02 0.03 0.04 0.05 0.06 0.07 0.08 0.09
a;b;g-h,0.01 0.02 0.03 0.04 0.05 0.06 0.07 0.08 0.09
etc
```

Note that **`fundamental_matrix`** is a **`3×3`** matrix, flattened into a vector in row-major order.


### 1.3 OTHER NOTABLE THINGS
---

This competition is part of the <a href="https://image-matching-workshop.github.io/">Image Matching: Local Features and Beyond</a> workshop at <a href="https://cvpr2022.thecvf.com/">CVPR'22</a>. Selected submissions to the competition will be invited to give talks at the workshop on June 20, 2022 in New Orleans, USA. Attending the workshop is not required to participate in the competition, but only teams that are attending the workshop will be considered to present their work.

CVPR 2022 will be a hybrid conference. Attendees presenting in person are responsible for all costs associated with expenses and fees to attend CVPR 2022.

### 1.4 DATASET OVERVIEW

---

<br><b style="text-decoration: underline; font-family: Verdana; text-transform: uppercase;">GENERAL INFORMATION</b>

Aligning photographs of the same scene is a problem of longstanding interest to computer vision researchers. Your challenge in this competition is to generate mappings between pairs of photos from various cities.

This competition uses a hidden test (n~=10_000). 
* When your submitted notebook is scored, the actual test data (including a sample submission) will be made available to your notebook.

<br><b style="text-decoration: underline; font-family: Verdana; text-transform: uppercase;">FILE INFORMATION</b>

<code style="font-weight: bold; font-size: 14px;">train/*/calibration.csv</code>
* **`image_id`**
    * The image filename.
* **`camera_intrinsics`** 
    * The 3×3 calibration matrix 𝐊 for this image, flattened into a vector by row-major indexing.
* **`rotation_matrix`**
    * The 3×3 rotation matrix 𝐑 for this image, flattened into a vector by row-major indexing.
* **`translation_vector`**
    * The translation vector 𝐓.

<code style="font-weight: bold; font-size: 14px;">train/*/pair_covisibility.csv</code>
* **`pair`**
    * A string identifying a pair of images, encoded as two image filenames (without the extension) separated by a hyphen, as **`key1-key2`**, where **`key1 > key2`**.
* **`covisibility`**
    * An estimate of the overlap between the two images. 
    * Higher numbers indicate greater overlap. 
    * We recommend using all pairs with a covisibility estimate of 0.1 or above. 
    * The procedure used to derive this number is described in Section 3.2 and Figure 5 of <a href="https://arxiv.org/pdf/2003.01587.pdf">this paper</a>.
* **`fundamental_matrix1`**
    * **The target column** as derived from the calibration files. 
    * Please see the <a href="https://www.kaggle.com/competitions/image-matching-challenge-2022/overview/problem-definition">problem definition page</a> for more details.

<code style="font-weight: bold; font-size: 14px;">train/scaling_factors.csv</code> 
* The poses for each scene where reconstructed via <a href="https://en.wikipedia.org/wiki/Structure_from_motion">Structure-from-Motion</a>, and are only accurate up to a scaling factor. 
* This file contains a scalar for each scene which can be used to convert them to meters. 
* For code examples, please refer to <a href="https://www.kaggle.com/eduardtrulls/imc2022-tutorial-load-and-evaluate-training-data">this notebook</a>.

<code style="font-weight: bold; font-size: 14px;">train/*/images/</code>
* A batch of images all taken near the same location.

<code style="font-weight: bold; font-size: 14px;">train/LICENSE.txt</code>
* Records of the specific source of and license for each image.

<code style="font-weight: bold; font-size: 14px;">sample_submission.csv</code>
* A valid sample submission.
* **`sample_id`**
    * The unique identifier for the image pair.
* **`fundamental_matrix`**
    * The target column. Please see the <a href="https://www.kaggle.com/competitions/image-matching-challenge-2022/overview/problem-definition">problem definition page</a> for more details. 
    * The default values are randomly generated.

<code style="font-weight: bold; font-size: 14px;">test.csv</code> 
* Expect to see roughly 10,000 pairs of images in the hidden test set.
* **`sample_id`**
    * The unique identifier for the image pair.
* **`batch_id`**
    * The batch ID.
* **`image_[1/2]_id`**
    * The filenames of each image in the pair.

<code style="font-weight: bold; font-size: 14px;">test_images</code> 
The test set. 
* The test data comes from a different source than the train data and contains photos of mostly urban scenes with variable degrees of overlap. 
* The two images forming a pair may have been collected months or years apart, but **never less than 24 hours.**
* Bridging this domain gap is part of the competition. 
* The images have been resized so that the **longest edge is around 800 pixels**, **may have different aspect ratios (including portrait and landscape), and are upright**.


## 2. SETUP
---


### 2.1 ACCELERATOR DETECTION
---

In order to use **`TPU`**, we use **`TPUClusterResolver`** for the initialization which is necessary to connect to the remote cluster and initialize cloud TPUs. Let's go over two important points

1. When using TPU on Kaggle, you don't need to specify arguments for **`TPUClusterResolver`**
2. However, on **G**oogle **C**ompute **E**ngine (**GCE**), you will need to do the following:

<br>

```python
# The name you gave to the TPU to use
TPU_WORKER = 'my-tpu-name'

# or you can also specify the grpc path directly
# TPU_WORKER = 'grpc://xxx.xxx.xxx.xxx:8470'

# The zone you chose when you created the TPU to use on GCP.
ZONE = 'us-east1-b'

# The name of the GCP project where you created the TPU to use on GCP.
PROJECT = 'my-tpu-project'

tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu=TPU_WORKER, zone=ZONE, project=PROJECT)
```

**WARNING:**


Although the Tensorflow documentation says it is the <b>project name</b> that should be provided for the argument <b><code>`project`</code></b>, it is actually the <b>Project ID</b>, that you should provide. This can be found on the GCP project dashboard page.

**REFERENCES:**


- [ Guide - Use TPUs](https://www.tensorflow.org/guide/tpu#tpu_initialization)
- [Doc - TPUClusterResolver](https://www.tensorflow.org/api_docs/python/tf/distribute/cluster_resolver/TPUClusterResolver)


In [ ]:
print(f"\n... ACCELERATOR SETUP STARTING ...\n")

# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    TPU = tf.distribute.cluster_resolver.TPUClusterResolver()  
except ValueError:
    TPU = None

if TPU:
    print(f"\n... RUNNING ON TPU - {TPU.master()}...")
    tf.config.experimental_connect_to_cluster(TPU)
    tf.tpu.experimental.initialize_tpu_system(TPU)
    strategy = tf.distribute.experimental.TPUStrategy(TPU)
else:
    print(f"\n... RUNNING ON CPU/GPU ...")
    # Yield the default distribution strategy in Tensorflow
    #   --> Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy() 

# What Is a Replica?
#    --> A single Cloud TPU device consists of FOUR chips, each of which has TWO TPU cores. 
#    --> Therefore, for efficient utilization of Cloud TPU, a program should make use of each of the EIGHT (4x2) cores. 
#    --> Each replica is essentially a copy of the training graph that is run on each core and 
#        trains a mini-batch containing 1/8th of the overall batch size
N_REPLICAS = strategy.num_replicas_in_sync
    
print(f"... # OF REPLICAS: {N_REPLICAS} ...\n")

print(f"\n... ACCELERATOR SETUP COMPLTED ...\n")

### 2.2 COMPETITION DATA ACCESS
---

TPUs read data must be read directly from **G**oogle **C**loud **S**torage **(GCS)**. Kaggle provides a utility library – **`KaggleDatasets`** – which has a utility function **`.get_gcs_path`** that will allow us to access the location of our input datasets within **GCS**.<br><br>

**TIPS:**


If you have multiple datasets attached to the notebook, you should pass the name of a specific dataset to the <b><code>`get_gcs_path()`</code></b> function. <i>In our case, the name of the dataset is the name of the directory the dataset is mounted within.</i><br><br>

In [ ]:
print("\n... DATA ACCESS SETUP STARTED ...\n")

if TPU:
    # Google Cloud Dataset path to training and validation images
    DATA_DIR = KaggleDatasets().get_gcs_path('image-matching-challenge-2022')
    save_locally = tf.saved_model.SaveOptions(experimental_io_device='/job:localhost')
    load_locally = tf.saved_model.LoadOptions(experimental_io_device='/job:localhost')
else:
    # Local path to training and validation images
    DATA_DIR = "/kaggle/input/image-matching-challenge-2022"
    save_locally = None
    load_locally = None

print(f"\n... DATA DIRECTORY PATH IS:\n\t--> {DATA_DIR}")

print(f"\n... IMMEDIATE CONTENTS OF DATA DIRECTORY IS:")
for file in tf.io.gfile.glob(os.path.join(DATA_DIR, "*")): print(f"\t--> {file}")

print("\n\n... DATA ACCESS SETUP COMPLETED ...\n")

### 2.3 LEVERAGING XLA OPTIMIZATIONS

---
**XLA** (Accelerated Linear Algebra) is a domain-specific compiler for linear algebra that can accelerate TensorFlow models with potentially no source code changes. **The results are improvements in speed and memory usage**.

<br>

When a TensorFlow program is run, all of the operations are executed individually by the TensorFlow executor. Each TensorFlow operation has a precompiled GPU/TPU kernel implementation that the executor dispatches to.

XLA provides us with an alternative mode of running models: it compiles the TensorFlow graph into a sequence of computation kernels generated specifically for the given model. Because these kernels are unique to the model, they can exploit model-specific information for optimization.<br><br>


**WARNING:**

XLA can not currently compile functions where dimensions are not inferrable: that is, if it's not possible to infer the dimensions of all tensors without running the entire computation

**NOTE:**

XLA compilation is only applied to code that is compiled into a graph (in <b>TF2</b> that's only a code inside <b><code>tf.function</code></b>).<br>- The <b><code>jit_compile</code></b> API has must-compile semantics, i.e. either the entire function is compiled with XLA, or an <b><code>errors.InvalidArgumentError</code></b> exception is thrown)

**REFERENCE:**

- [XLA: Optimizing Compiler for Machine Learning](https://www.tensorflow.org/xla)

In [ ]:
print(f"\n... XLA OPTIMIZATIONS STARTING ...\n")

print(f"\n... CONFIGURE JIT (JUST IN TIME) COMPILATION ...\n")
# enable XLA optmizations (10% speedup when using @tf.function calls)
tf.config.optimizer.set_jit(True)

print(f"\n... XLA OPTIMIZATIONS COMPLETED ...\n")

### 2.4 BASIC DATA DEFINITIONS & INITIALIZATIONS
---

In [ ]:
print("\n... BASIC DATA SETUP STARTING ...\n\n")

print("\n... TRAIN META ...\n")
TRAIN_DIR = os.path.join(DATA_DIR, "train")
SCALING_CSV = os.path.join(TRAIN_DIR, "scaling_factors.csv")
scaling_df = pd.read_csv(SCALING_CSV)
scale_map = scaling_df.groupby("scene")["scaling_factor"].first().to_dict()

# 'british_museum', 'piazza_san_marco', 
# 'trevi_fountain', 'st_pauls_cathedral', 
# 'colosseum_exterior', 'buckingham_palace', 
# 'temple_nara_japan', 'sagrada_familia', 
# 'grand_place_brussels', 'pantheon_exterior', 
# 'notre_dame_front_facade', 'st_peters_square', 
# 'sacre_coeur', 'taj_mahal', 
# 'lincoln_memorial_statue', 'brandenburg_gate'
TRAIN_SCENES = scaling_df.scene.unique().tolist()

train_map = {} 
for _s in tqdm(TRAIN_SCENES, total=len(TRAIN_SCENES)):
    # Initialize    
    train_map[_s] = {}
    
    # Image Stuff
    train_map[_s]["images"] = sorted(tf.io.gfile.glob(os.path.join(TRAIN_DIR, _s, "images", "*.jpg")))
    train_map[_s]["image_ids"] = [_f_path[:-4].rsplit("/", 1)[-1] for _f_path in train_map[_s]["images"]]
    
    # Calibration Stuff (CAL)
    _tmp_cal_df = pd.read_csv(os.path.join(TRAIN_DIR, _s, "calibration.csv"))
    _tmp_cal_df["image_path"] = os.path.join(TRAIN_DIR, _s, "images")+"/"+_tmp_cal_df["image_id"]+".jpg"
    train_map[_s]["cal_df"]=_tmp_cal_df.copy()
        
    # Pair Covisibility Stuff (PCO)
    _tmp_pco_df = pd.read_csv(os.path.join(TRAIN_DIR, _s, "pair_covisibility.csv"))
    _tmp_pco_df["image_id_1"], _tmp_pco_df["image_id_2"] = zip(*_tmp_pco_df.pair.apply(lambda x: x.split("-")))
    _tmp_pco_df["image_path_1"] = os.path.join(TRAIN_DIR, _s, "images")+"/"+_tmp_pco_df["image_id_1"]+".jpg"
    _tmp_pco_df["image_path_2"] = os.path.join(TRAIN_DIR, _s, "images")+"/"+_tmp_pco_df["image_id_2"]+".jpg"
    train_map[_s]["pco_df"] = _tmp_pco_df.copy()

#cleanup
del _tmp_cal_df, _tmp_pco_df; gc.collect(); gc.collect();
    
print("\n... TEST META ...\n")
TEST_IMG_DIR = os.path.join(DATA_DIR, "test_images")
TEST_CSV = os.path.join(DATA_DIR, "test.csv")
test_df = pd.read_csv(TEST_CSV)
test_df["f_path_1"] = TEST_IMG_DIR+"/"+test_df.batch_id+"/"+test_df.image_1_id+".png"
test_df["f_path_2"] = TEST_IMG_DIR+"/"+test_df.batch_id+"/"+test_df.image_2_id+".png"
display(test_df)

SS_CSV = os.path.join(DATA_DIR, "sample_submission.csv")
ss_df = pd.read_csv(SS_CSV)
display(ss_df)

## 3. HELPER FUNCTION
---

In [ ]:
def flatten_l_o_l(nested_list):
    """ Flatten a list of lists """
    nested_list = [x if type(x) is list else [x,] for x in nested_list]
    return [item for sublist in nested_list for item in sublist]

def plot_two_paths(f_path_1, f_path_2, _titles=None):
    plt.figure(figsize=(20, 10))
    
    plt.subplot(1,2,1)
    if _titles is None:
        plt.title(f_path_1, fontweight="bold")
    else:
        plt.title(_titles[0], fontweight="bold")
    plt.imshow(cv2.imread(f_path_1)[..., ::-1])
    plt.axis(False)
    
    plt.subplot(1,2,2)
    if _titles is None:
        plt.title(f_path_2, fontweight="bold")
    else:
        plt.title(_titles[1], fontweight="bold")

    plt.imshow(cv2.imread(f_path_2)[..., ::-1])
    plt.axis(False)
    
    plt.tight_layout()
    plt.show()
    
def pco_random_plot(_df):
    df_row = _df.sample(1).reset_index(drop=True)
    plot_two_paths(
        df_row.image_path_1[0],
        df_row.image_path_2[0],
        [f"Image ID: {df_row.image_id_1} - COV={df_row.covisibility[0]}",
         f"Image ID: {df_row.image_id_2} - COV={df_row.covisibility[0]}",]
    )
    
def arr_from_str(_s):
    return np.fromstring(_s, sep=" ").reshape((-1,3)).squeeze()

def get_id2cal_map(_train_map):
    _id2cal_map = {}
    for _s in tqdm(TRAIN_SCENES, total=len(TRAIN_SCENES)):
        for _, _row in _train_map[_s]["cal_df"].iterrows():
            _img_id = _row["image_id"]; _id2cal_map[_img_id]={}
            _id2cal_map[_img_id]["camera_intrinsics"] = arr_from_str(_row["camera_intrinsics"])
            _id2cal_map[_img_id]["rotation_matrix"] = arr_from_str(_row["rotation_matrix"])
            _id2cal_map[_img_id]["translation_vector"] = arr_from_str(_row["translation_vector"])
            _id2cal_map[_img_id]["image_path"] = _row["image_path"]
    return _id2cal_map

id2cal_map = get_id2cal_map(train_map)

**The following functions are for feature identification, matrix manipulations, etc**

In [ ]:
def extract_sift_features(rgb_image, detector, n_features):
    """ 
    Helper Function to Compute SIFT features for a Given Image
    REFERENCE --> https://www.kaggle.com/code/eduardtrulls/imc2022-training-data?scriptVersionId=92062607
    """
    # Convert RGB image to Grayscale
    gray = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2GRAY)
    
    # Run detector and retrieve only the top keypoints and descriptors 
    kp, desc = detector.detectAndCompute(gray, None)
    
    return kp[:n_features], desc[:n_features]

def extract_keypoints(rgb_image, detector):
    """ 
    Helper Function to Compute SIFT features for a Given Image
    REFERENCE --> https://www.kaggle.com/code/eduardtrulls/imc2022-training-data?scriptVersionId=92062607
    """
    # Convert RGB image to Grayscale
    gray = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2GRAY)
    
    # Run detector and retrieve the keypoints
    return detector.detect(gray)

def build_composite_image(im1, im2, axis=1, margin=0, background=1):
    """
    Convenience function to stack two images with different sizes.
    REFERENCE --> https://www.kaggle.com/code/eduardtrulls/imc2022-training-data?scriptVersionId=92062607
    """
    
    if background != 0 and background != 1:
        background = 1
    if axis != 0 and axis != 1:
        raise RuntimeError('Axis must be 0 (vertical) or 1 (horizontal')

    h1, w1, _ = im1.shape
    h2, w2, _ = im2.shape

    if axis == 1:
        composite = np.zeros((max(h1, h2), w1 + w2 + margin, 3), dtype=np.uint8) + 255 * background
        if h1 > h2:
            voff1, voff2 = 0, (h1 - h2) // 2
        else:
            voff1, voff2 = (h2 - h1) // 2, 0
        hoff1, hoff2 = 0, w1 + margin
    else:
        composite = np.zeros((h1 + h2 + margin, max(w1, w2), 3), dtype=np.uint8) + 255 * background
        if w1 > w2:
            hoff1, hoff2 = 0, (w1 - w2) // 2
        else:
            hoff1, hoff2 = (w2 - w1) // 2, 0
        voff1, voff2 = 0, h1 + margin
    composite[voff1:voff1 + h1, hoff1:hoff1 + w1, :] = im1
    composite[voff2:voff2 + h2, hoff2:hoff2 + w2, :] = im2

    return (composite, (voff1, voff2), (hoff1, hoff2))


def draw_cv_matches(im1, im2, kp1, kp2, matches, axis=1, margin=0, background=0, linewidth=2):
    """
    Draw keypoints and matches.
    REFERENCE --> https://www.kaggle.com/code/eduardtrulls/imc2022-training-data?scriptVersionId=92062607
    """
    
    composite, v_offset, h_offset = build_composite_image(im1, im2, axis, margin, background)

    # Draw all keypoints.
    for coord_a, coord_b in zip(kp1, kp2):
        composite = cv2.drawMarker(composite, (int(coord_a[0] + h_offset[0]), int(coord_a[1] + v_offset[0])), color=(255, 0, 0), markerType=cv2.MARKER_CROSS, markerSize=5, thickness=1)
        composite = cv2.drawMarker(composite, (int(coord_b[0] + h_offset[1]), int(coord_b[1] + v_offset[1])), color=(255, 0, 0), markerType=cv2.MARKER_CROSS, markerSize=5, thickness=1)
    
    # Draw matches, and highlight keypoints used in matches.
    for idx_a, idx_b in matches:
        composite = cv2.drawMarker(composite, (int(kp1[idx_a, 0] + h_offset[0]), int(kp1[idx_a, 1] + v_offset[0])), color=(0, 0, 255), markerType=cv2.MARKER_CROSS, markerSize=12, thickness=1)
        composite = cv2.drawMarker(composite, (int(kp2[idx_b, 0] + h_offset[1]), int(kp2[idx_b, 1] + v_offset[1])), color=(0, 0, 255), markerType=cv2.MARKER_CROSS, markerSize=12, thickness=1)
        composite = cv2.line(composite,
                             tuple([int(kp1[idx_a][0] + h_offset[0]),
                                   int(kp1[idx_a][1] + v_offset[0])]),
                             tuple([int(kp2[idx_b][0] + h_offset[1]),
                                   int(kp2[idx_b][1] + v_offset[1])]), color=(0, 0, 255), thickness=1)
    return composite

def normalize_keypoints(keypoints, K):
    C_x = K[0, 2]
    C_y = K[1, 2]
    f_x = K[0, 0]
    f_y = K[1, 1]
    keypoints = (keypoints - np.array([[C_x, C_y]])) / np.array([[f_x, f_y]])
    return keypoints

def compute_essential_matrix(F, K1, K2, kp1, kp2):
    """
    Compute the Essential matrix from the Fundamental matrix, given the calibration matrices. 
    Note that we ask participants to estimate F, i.e., without relying on known intrinsics.
    REFERENCE --> https://www.kaggle.com/code/eduardtrulls/imc2022-training-data?scriptVersionId=92062607
    """
    
    # Warning! Old versions of OpenCV's RANSAC could return multiple F matrices, encoded as a single matrix size 6x3 or 9x3, rather than 3x3.
    # We do not account for this here, as the modern RANSACs do not do this:
    # https://opencv.org/evaluating-opencvs-new-ransacs
    assert F.shape[0] == 3, 'Malformed F?'

    # Use OpenCV's recoverPose to solve the cheirality check:
    # https://docs.opencv.org/4.5.4/d9/d0c/group__calib3d.html#gadb7d2dfcc184c1d2f496d8639f4371c0
    E = np.matmul(np.matmul(K2.T, F), K1).astype(np.float64)
    
    kp1n = normalize_keypoints(kp1, K1)
    kp2n = normalize_keypoints(kp2, K2)
    num_inliers, R, T, mask = cv2.recoverPose(E, kp1n, kp2n)

    return E, R, T

def quaternion_from_matrix(matrix):
    """
    Transform a rotation matrix into a quaternion
    REFERENCE --> https://www.kaggle.com/code/eduardtrulls/imc2022-training-data?scriptVersionId=92062607
    
    The quaternion is the eigenvector of K that corresponds to the largest eigenvalue.\
    """

    M = np.array(matrix, dtype=np.float64, copy=False)[:4, :4]
    m00 = M[0, 0]
    m01 = M[0, 1]
    m02 = M[0, 2]
    m10 = M[1, 0]
    m11 = M[1, 1]
    m12 = M[1, 2]
    m20 = M[2, 0]
    m21 = M[2, 1]
    m22 = M[2, 2]

    K = np.array([
        [m00 - m11 - m22, 0.0, 0.0, 0.0],
        [m01 + m10, m11 - m00 - m22, 0.0, 0.0],
        [m02 + m20, m12 + m21, m22 - m00 - m11, 0.0],
        [m21 - m12, m02 - m20, m10 - m01, m00 + m11 + m22]
    ])
    K /= 3.0

    # The quaternion is the eigenvector of K that corresponds to the largest eigenvalue.
    w, V = np.linalg.eigh(K)
    q = V[[3, 0, 1, 2], np.argmax(w)]
    if q[0] < 0:
        np.negative(q, q)
    return q


def compute_error_for_example(q_gt, T_gt, q, T, scale, eps=1e-15):
    '''Compute the error metric for a single example.
    
    The function returns two errors, over rotation and translation. 
    These are combined at different thresholds by ComputeMaa in order to compute the mean Average Accuracy.'''
    
    q_gt_norm = q_gt / (np.linalg.norm(q_gt) + eps)
    q_norm = q / (np.linalg.norm(q) + eps)

    loss_q = np.maximum(eps, (1.0 - np.sum(q_norm * q_gt_norm)**2))
    err_q = np.arccos(1 - 2 * loss_q)

    # Apply the scaling factor for this scene.
    T_gt_scaled = T_gt * scale
    T_scaled = T * np.linalg.norm(T_gt) * scale / (np.linalg.norm(T) + eps)

    err_t = min(np.linalg.norm(T_gt_scaled - T_scaled), np.linalg.norm(T_gt_scaled + T_scaled))

    return err_q * 180 / np.pi, err_t

## 4. DATASET EXPLORATION & PREPROCESSING
---

Let's investigate the provided information and then plot some images for various things

### 4.1 THE TRAINING DATA
---

Before we can make some simple deductions, we need to combine the data as much as possible so we can view it with a comprehensive lense.

In [ ]:
_cal_dfs = []
_pco_dfs = []
_s_dfs = []
for _s in TRAIN_SCENES:
    _s_map = train_map[_s]
    _s_dfs.append(pd.DataFrame({
        "scene":[_s,]*len(_s_map["images"]),
        "f_path":_s_map["images"],
    }))
    _cal_dfs.append(_s_map["cal_df"])
    _pco_dfs.append(_s_map["pco_df"])

all_pco_dfs = pd.concat(_pco_dfs).reset_index(drop=True)
all_cal_dfs = pd.concat(_cal_dfs).reset_index(drop=True)

train_df = pd.concat(_s_dfs).reset_index(drop=True)
train_df.insert(0, "image_id", train_df.f_path.apply(lambda x: x[:-4].rsplit("/", 1)[-1]))
train_df = pd.merge(train_df, all_cal_dfs, on="image_id").drop(columns=["image_path",])

cov_1_df = all_pco_dfs[["image_id_1", "covisibility"]]
cov_1_df.columns = ["image_id", "covisibility"]
cov_2_df = all_pco_dfs[["image_id_2", "covisibility"]]
cov_2_df.columns = ["image_id", "covisibility"]
cov_df = pd.concat([cov_1_df,cov_2_df])
img_id_2_cov = cov_df.groupby("image_id")["covisibility"].mean().to_dict()
del cov_1_df, cov_2_df, cov_df; gc.collect();

# Add a column for average covisibility
#    - i.e. For a given image, what is the average 
#           covisibility of all of it's respective pairs
train_df.insert(2, "mean_covisibility", train_df.image_id.map(img_id_2_cov))
train_df

In [ ]:
fig = px.histogram(train_df, "scene", color="scene", title="<b>Image Counts By Scene</b>")
fig.update_layout(
    yaxis_title="<b>Number Of Images</b>", xaxis_title="<b>Scene Identifier</b>",
    legend_title="<b>Scene Identifier</b>", showlegend=False,
)
fig.show()

fig = px.histogram(train_df, "mean_covisibility", color="scene", title="<b>Average Covisibility Of Images Coloured By Scene</b>", )
fig.update_layout(
    yaxis_title="<b>Number Of Images Per Covisibility Bin</b>", xaxis_title="<b>Covisibility (0-1.00)</b>",
    legend_title="<b>Scene Identifier</b>",
)
fig.show()

### 4.2 LET'S LOOK INTO THE CAMERA MATRIX
---

- [REFERENCE](https://www.wikiwand.com/en/Camera_matrix) 
The calibration matrices for a given camera calculate the camera matrix using the extrinsic and intrinsic parameters. The extrinsic parameters represent a rigid transformation from 3-D world coordinate system to the 3-D camera’s coordinate system. The intrinsic parameters represent a projective transformation from the 3-D camera’s coordinates into the 2-D image coordinates.

---

This is a fancy way of saying that **extrinsics** 

* describe the cameras location within the external (outside-the-camera) world 
    * <b>$(x,y,z)$</b>
* describe the cameras orientation with respect to the external (outside-the-camera) world 
    * <b>$(\gamma, \beta, \alpha)$</b>
* Note while these are the values that describe the absolute positions/rotations of the camera, it is often useful to represent them as a series of operations that would move a camera from a level position at the origin to whatever respective position the camera exists at. This is where the concepts of the **rotation** and **translation** matrix come from.
    
while **intrinsics**...

* Describe how a point in the 3D world is mapped to the 2D image plane
    * focal length
    * skew/distortion/scale
    * principal offsets
    * etc.

We can take this to the logical conclusion of building a matrix that can fully describe a camera (the union of these two matrices). This new matrix is called **the camera matrix**.

---


In computer vision a camera matrix or (camera) projection matrix is a $3\times 4$ matrix which describes the mapping of a pinhole camera from 3D points in the world to 2D points in an image.

Let $\mathbf {x}$  be a representation of a 3D point in homogeneous coordinates (a 4-dimensional vector), and let  $\mathbf{y}$  be a representation of the image of this point in the pinhole camera (a 3-dimensional vector). Then the following relation holds

$$
 \mathbf{y} \sim \mathbf{C} \, \mathbf{x} 
$$

where  $\mathbf{C}$  is the camera matrix and the $\, \sim$  sign implies that the left and right hand sides are equal up to a non-zero scalar multiplication. Since the camera matrix  $\mathbf{C}$  is involved in the mapping between elements of two projective spaces, it too can be regarded as a projective element. 

**This means that  $\mathbf{C}$  has only 11 degrees of freedom since any multiplication by a non-zero scalar results in an equivalent camera matrix.**

---

***I will spare you the derivation and normalization steps... if you're interested they are detailed in the wiki page linked in reference one above.***

---

Given the mapping produced by a normalized camera matrix (skipped), the resulting normalized image coordinates can be transformed by means of an arbitrary 2D homography. This **includes 2D translations and rotations** as well as **scaling** (**isotropic** and **anisotropic**) but also general 2D perspective transformations. 

Such a transformation can be represented as a $3\times  3$  matrix  $\mathbf {H}$  which maps the homogeneous normalized image coordinates  $\mathbf{y}$  to the homogeneous transformed image coordinates ${\mathbf  {y}}'$:

$$
{\mathbf  {y}}'={\mathbf  {H}}\,{\mathbf  {y}}
$$

Inserting the above expression for the normalized image coordinates in terms of the 3D coordinates gives

$$
{\mathbf  {y}}'={\mathbf  {H}}\,{\mathbf  {C}}_N\,{\mathbf  {x}}'
$$

This produces the most general form of camera matrix (Note we sometimes use $\mathbf {K}$ instead of $\mathbf {H}$)

$$
{\mathbf  {C}}={\mathbf  {H}}\,{\mathbf  {C}}_N\,{\mathbf  {x}}'={\mathbf  {H} \left( {\mathbf {R}} \, \middle| \, {\mathbf {t}} \right)}
$$

### 4.3 LET'S LOOK INTO THE CAMERA INSTRICS
---
- [REFERENCE](https://ksimek.github.io/2013/08/13/intrinsic)

The intrinsic matrix is parameterized by Hartley and Zisserman as

$$K=\begin{pmatrix}
f_x&s&x_0\\
0&f_y&y_0\\
0&0&1
\end{pmatrix}$$

Each intrinsic parameter describes a geometric property of the camera. 

Let's examine each of these properties in detail.

<br>

**Focal Length** –– $(f_x, f_y)$

The focal length is the distance between the pinhole and the film (a.k.a. image plane). 
* For reasons... the focal length is measured in pixels. 
* In a true pinhole camera, both $f_x$ and $f_y$ have the same value, which is illustrated as $f$ below.
    * In the code cell below we verify that for all our examples $f_x = f_y$
* In practice, $f_x$ and $f_y$ can differ for a number of reasons... however, we will ignore them for now as in our dataset... these two values are always equal (assume perfectly square pixels!)

<center><img src="https://ksimek.github.io/img/intrinsic-focal-length.png"></center>

<br>

**Principal Point Offset** –– $(x_0, y_0)$

The camera's **"principal axis"** is the line perpendicular to the image plane that passes through the pinhole. 
* Its itersection with the image plane is referred to as the **"principal point"**, illustrated below.

<center><img src="https://ksimek.github.io/img/intrinsic-pp.png"></center>

The **"principal point offset"** is the location of the principal point relative to **the film's origin**. 
* The exact definition depends on which convention is used for the location of the origin
* the illustration below assumes it's at the bottom-left of the film.

<center><img src="https://ksimek.github.io/img/intrinsic-pp-offset.png"></center>

Increasing $x_0$ shifts the pinhole to the right

<center><img src="https://ksimek.github.io/img/intrinsic-pp-offset-delta-alt.png"></center>

This is equivalent to shifting the film to the left and leaving the pinhole unchanged.

<center><img src="https://ksimek.github.io/img/intrinsic-pp-offset-delta.png"></center>

Notice that the box surrounding the camera is irrelevant, only the pinhole's position relative to the film matters.

<br>

**Axis Skew** –– $(s)$

Axis skew causes shear distortion in the projected image. 
* In our dataset all skew values should be 0... but it's worth noting that some digitization processes can cause nonzero skew

<br>

---

We can decompose the intrinsic matrix into a sequence of shear, scaling, and translation transformations, corresponding to axis skew, focal length, and principal point offset, respectively:

<center><img src="https://i.ibb.co/Gvt4CBx/Screen-Shot-2022-04-11-at-1-38-20-PM.png"></center>

---

Now that we understand the basics of the camera intrinsic matrix, we can investigate the distribution of the respective values within the provided train dataset. We will first extract the respective information entities from the intrinsics matrix, verify certain assumptions, and then plot the relevant distributions.

In [ ]:
# NOTE: Our intrinsics matrix is a flat string... NOTE: [idx]
#       but when reshaped to 3x3 it will map as follows...
# STRING --> "TL[0] T[1] TR[2] CL[3] C[4] CR[5] BL[6] B[7] BR[8]"
#
# MATRIX BELOW
##############
#            #
#  TL  T  TR #
#  CL  C  CR #
#  BL  B  BR #
#            #
##############
# 
# Therefore we just index into the string and convert to a digit to access certain elements
train_df["fx"] = train_df.camera_intrinsics.apply(lambda x: float(x.split()[0])) # 0 = TL 
train_df["fy"] = train_df.camera_intrinsics.apply(lambda x: float(x.split()[4])) # 4 = C
train_df["x0"] = train_df.camera_intrinsics.apply(lambda x: float(x.split()[2])) # 0 = TR
train_df["y0"] = train_df.camera_intrinsics.apply(lambda x: float(x.split()[5])) # 4 = CR
train_df["s"] = train_df.camera_intrinsics.apply(lambda x: float(x.split()[1])) # 1 = TC

if (train_df["fx"]!=train_df["fy"]).sum()==0:
    print("All fx values (focal length x) are equal to the respective fy values (focal length y)... as expected")
    
if train_df["s"].sum()==0:
    print("All skew values are 0... as expected")

In [ ]:
fig = px.histogram(train_df, "x0", title="<b>Principal Point Offset [x<sub>0</sub>] Coloured By Scene</b>", color="scene", log_y=False)
fig.update_layout(
    yaxis_title="<b>Number Of Images Per x<sub>0</sub> Bin</b>", xaxis_title="<b>Principal Point Offset [x<sub>0</sub>]</b>",
    legend_title="<b>Scene Identifier</b>",
)
fig.show()

fig = px.histogram(train_df, "y0", title="<b>Principal Point Offset [y<sub>0</sub>] Coloured By Scene</b>", color="scene", log_y=False)
fig.update_layout(
    yaxis_title="<b>Number Of Images Per y<sub>0</sub> Bin</b>", xaxis_title="<b>Principal Point Offset [y<sub>0</sub>]</b>",
    legend_title="<b>Scene Identifier</b>",
)
fig.show()

# We use "fx" because all "fx" values are equivalent to "fy" values
fig = px.histogram(train_df, "fx", title="<b>Focal Lengths Coloured By Scene</b>", color="scene", log_x=True)
fig.update_layout(
    yaxis_title="<b>Number Of Images Per Focal Length Bin</b>", xaxis_title="<b>Focal Length <i>(log scale)</i></b>",
    legend_title="<b>Scene Identifier</b>",
)
fig.show()

### 4.4 LET'S LOOK INTO THE CAMERA EXTRINSICS
---
- [REFERENCE](https://ksimek.github.io/2012/08/22/extrinsic)

As discussed previously, the camera's **extrinsic matrix** describes the **camera's location in the world (3D), and what direction it's pointing (orientation)**. It has two components: 
* **a rotation matrix, $\mathbf{R}$ (or $\theta$)**
* **a translation vector $\mathbf{t}$**

However, shocker, these don't exactly correspond to the camera's rotation and translation. 

The extrinsic matrix takes the form of a rigid transformation matrix: 
* a $3\times 3$ rotation matrix in the left-block
* a $3\times 1$ translation column-vector in the right

$$
\left [
    \begin{array}{c|c}
R & t
    \end{array}
\right] = 
\left [
    \begin{array}{ccc|c}
r_{1,1} & r_{1,2} & r_{1,3} & t_1 \\
r_{2,1} & r_{2,2} & r_{2,3} & t_2 \\
r_{3,1} & r_{3,2} & r_{3,3} & t_3
    \end{array}
\right]
$$

<br>

It's common to see a version of this matrix with extra row of $(0,0,0,1)$ added to the bottom. This makes the matrix square, which allows us to further decompose this matrix into a rotation followed by translation

<br>

$$
\begin{align}
    \left [
        \begin{array}{c|c} 
            R & \boldsymbol{t} \\
            \hline
            \boldsymbol{0} & 1 
        \end{array}
    \right ] &= 
    \left [
        \begin{array}{c|c} 
            I & \boldsymbol{t} \\
            \hline
            \boldsymbol{0} & 1 
        \end{array}
    \right ] 
    \times
    \left [
        \begin{array}{c|c} 
            R & \boldsymbol{0} \\
            \hline
            \boldsymbol{0} & 1 
        \end{array}
    \right ] \\
        &=
\left[ \begin{array}{ccc|c} 
1 & 0 & 0 & t_1 \\
0 & 1 & 0 & t_2 \\
0 & 0 & 1 & t_3 \\
  \hline
0 & 0 & 0 & 1
\end{array} \right] \times
\left[ \begin{array}{ccc|c} 
r_{1,1} & r_{1,2} & r_{1,3} & 0  \\
r_{2,1} & r_{2,2} & r_{2,3} & 0 \\
r_{3,1} & r_{3,2} & r_{3,3} & 0 \\
  \hline
0 & 0 & 0 & 1
\end{array} \right] 
\end{align}
$$

<br>

This matrix describes how to transform points in world coordinates to camera coordinates. 
* The vector $t$ can be interpreted as the position of the world origin in camera coordinates
* The columns of $R$ represent represent the directions of the world-axes in camera coordinates.

**The important thing to remember about the extrinsic matrix is that it describes how the world is transformed relative to the camera.** This is often counter-intuitive, because we usually want to specify how the camera is transformed relative to the world. 

---

Let's briefly look into each matrix type to get a grasp of what is happening in each in laymens terms**

In 3D space, rotation can occur about the x, y, or z-axis. Such a type of rotation that occurs about any one of the axis is known as a basic or elementary rotation. Given below are the rotation matrices that can rotate a vector through an angle about any particular axis.

<br>

$$
\begin{align}
    R_x = \left [
        \begin{array}{ccc} 
            1 & 0 & 0 \\
            0 & \cos(\boldsymbol{\gamma}) & -\sin(\boldsymbol{\gamma}) \\
            0 & \sin(\boldsymbol{\gamma}) & \cos(\boldsymbol{\gamma}) 
        \end{array}
    \right ] 
\end{align} \qquad\text{This is also known as ROLL}\quad\text{(CC-rotation of }\gamma\text{ about the x-axis)}
$$ 

<br>

$$
\begin{align}
    R_y = \left [
        \begin{array}{ccc} 
            \cos(\boldsymbol{\beta}) & 0 & \sin(\boldsymbol{\beta}) \\
            0 & 1 & 0 \\
            -\sin(\boldsymbol{\beta}) & 0 & \cos(\boldsymbol{\beta})
        \end{array}
    \right ]
\end{align} \qquad\text{This is also known as PITCH}\quad\text{(CC-rotation of }\beta\text{ about the y-axis)}
$$

<br>

$$
\begin{align}
    R_z = \left [
        \begin{array}{ccc} 
            \cos(\boldsymbol{\alpha}) & -\sin(\boldsymbol{\alpha}) & 0 \\
            \sin(\boldsymbol{\alpha}) & \cos(\boldsymbol{\alpha}) & 0 \\
            0 & 0 & 1 
        \end{array}
    \right ]
\end{align} \qquad\text{This is also known as YAW}\quad\text{(CC-rotation of }\alpha\text{ about the z-axis)}
$$

In [ ]:
train_df["t_x"] = train_df.translation_vector.apply(lambda x: float(x.split()[0]))
train_df["t_y"] = train_df.translation_vector.apply(lambda x: float(x.split()[1]))
train_df["t_z"] = train_df.translation_vector.apply(lambda x: float(x.split()[2]))

train_df["r1_1"] = train_df.rotation_matrix.apply(lambda x: float(x.split()[0]))
train_df["r2_1"] = train_df.rotation_matrix.apply(lambda x: float(x.split()[1]))
train_df["r3_1"] = train_df.rotation_matrix.apply(lambda x: float(x.split()[2]))
train_df["r1_2"] = train_df.rotation_matrix.apply(lambda x: float(x.split()[3]))
train_df["r2_2"] = train_df.rotation_matrix.apply(lambda x: float(x.split()[4]))
train_df["r3_2"] = train_df.rotation_matrix.apply(lambda x: float(x.split()[5]))
train_df["r1_3"] = train_df.rotation_matrix.apply(lambda x: float(x.split()[6]))
train_df["r2_3"] = train_df.rotation_matrix.apply(lambda x: float(x.split()[7]))
train_df["r3_3"] = train_df.rotation_matrix.apply(lambda x: float(x.split()[8]))

fig = px.histogram(train_df, "t_x", title="<b>Translation Value <sub>x</sub> Coloured By Scene</b>", color="scene", log_y=True)
fig.update_layout(
    yaxis_title="<b>Number Of Images Per Bin <sub>(log-scale)</sub></b>", xaxis_title="<b>Translation Value <sub>x</sub></b>",
    legend_title="<b>Scene Identifier</b>",
)
fig.show()

fig = px.histogram(train_df, "t_y", title="<b>Translation Value <sub>y</sub> Coloured By Scene</b>", color="scene", log_y=True)
fig.update_layout(
    yaxis_title="<b>Number Of Images Per Bin <sub>(log-scale)</sub></b>", xaxis_title="<b>Translation Value <sub>y</sub></b>",
    legend_title="<b>Scene Identifier</b>",
)
fig.show()

fig = px.histogram(train_df, "t_z", title="<b>Translation Value <sub>z</sub> Coloured By Scene</b>", color="scene", log_y=True)
fig.update_layout(
    yaxis_title="<b>Number Of Images Per Bin <sub>(log-scale)</sub></b>", xaxis_title="<b>Translation Value <sub>z</sub></b>",
    legend_title="<b>Scene Identifier</b>",
)
fig.show()


fig = px.histogram(train_df, "r1_1", title="<b>Rotation Value <sub>1,1</sub> Coloured By Scene</b>", color="scene", log_y=True)
fig.update_layout(
    yaxis_title="<b>Number Of Images Per Bin <sub>(log-scale)</sub></b>", xaxis_title="<b>Rotation Value <sub>1,1</sub></b>",
    legend_title="<b>Scene Identifier</b>",
)
fig.show()


fig = px.histogram(train_df, "r2_1", title="<b>Rotation Value <sub>2,1</sub> Coloured By Scene</b>", color="scene", log_y=True)
fig.update_layout(
    yaxis_title="<b>Number Of Images Per Bin <sub>(log-scale)</sub></b>", xaxis_title="<b>Rotation Value <sub>2,1</sub></b>",
    legend_title="<b>Scene Identifier</b>",
)
fig.show()


fig = px.histogram(train_df, "r3_1", title="<b>Rotation Value <sub>3,1</sub> Coloured By Scene</b>", color="scene", log_y=True)
fig.update_layout(
    yaxis_title="<b>Number Of Images Per Bin <sub>(log-scale)</sub></b>", xaxis_title="<b>Rotation Value <sub>3,1</sub></b>",
    legend_title="<b>Scene Identifier</b>",
)
fig.show()


fig = px.histogram(train_df, "r1_2", title="<b>Rotation Value <sub>1,2</sub> Coloured By Scene</b>", color="scene", log_y=True)
fig.update_layout(
    yaxis_title="<b>Number Of Images Per Bin <sub>(log-scale)</sub></b>", xaxis_title="<b>Rotation Value <sub>1,2</sub></b>",
    legend_title="<b>Scene Identifier</b>",
)
fig.show()


fig = px.histogram(train_df, "r2_2", title="<b>Rotation Value <sub>2,2</sub> Coloured By Scene</b>", color="scene", log_y=True)
fig.update_layout(
    yaxis_title="<b>Number Of Images Per Bin <sub>(log-scale)</sub></b>", xaxis_title="<b>Rotation Value <sub>2,2</sub></b>",
    legend_title="<b>Scene Identifier</b>",
)
fig.show()


fig = px.histogram(train_df, "r3_2", title="<b>Rotation Value <sub>3,2</sub> Coloured By Scene</b>", color="scene", log_y=True)
fig.update_layout(
    yaxis_title="<b>Number Of Images Per Bin <sub>(log-scale)</sub></b>", xaxis_title="<b>Rotation Value <sub>3,2</sub></b>",
    legend_title="<b>Scene Identifier</b>",
)
fig.show()


fig = px.histogram(train_df, "r1_3", title="<b>Rotation Value <sub>1,3</sub> Coloured By Scene</b>", color="scene", log_y=True)
fig.update_layout(
    yaxis_title="<b>Number Of Images Per Bin <sub>(log-scale)</sub></b>", xaxis_title="<b>Rotation Value <sub>1,3</sub></b>",
    legend_title="<b>Scene Identifier</b>",
)
fig.show()


fig = px.histogram(train_df, "r2_3", title="<b>Rotation Value <sub>2,3</sub> Coloured By Scene</b>", color="scene", log_y=True)
fig.update_layout(
    yaxis_title="<b>Number Of Images Per Bin <sub>(log-scale)</sub></b>", xaxis_title="<b>Rotation Value <sub>2,3</sub></b>",
    legend_title="<b>Scene Identifier</b>",
)
fig.show()

fig = px.histogram(train_df, "r3_3", title="<b>Rotation Value <sub>3,3</sub> Coloured By Scene</b>", color="scene", log_y=True)
fig.update_layout(
    yaxis_title="<b>Number Of Images Per Bin <sub>(log-scale)</sub></b>", xaxis_title="<b>Rotation Value <sub>3,3</sub></b>",
    legend_title="<b>Scene Identifier</b>",
)
fig.show()

### 4.5 LOOK AT A SINGLE EXAMPLE W/ COVISIBILITY & CALIBRATION INFORMATION
---


In [ ]:
print("\nEXAMPLE OF HIGH COVISIBILITY")
plot_two_paths(
    train_map["british_museum"]["pco_df"]["image_path_1"][0], 
    train_map["british_museum"]["pco_df"]["image_path_2"][0],
    [f"Covisibility Between Is {train_map['british_museum']['pco_df'].covisibility[0]}",
     f"Covisibility Between Is {train_map['british_museum']['pco_df'].covisibility[0]}"]
)

print("\n\n\nIF WE EXAMINE THE CALIBRATION DATAFRAME ROW FOR THE TOP LEFT IMAGE (ID='93658023_4980549800') WE SEE THE FOLLOWING:")
cal_row_from_id = train_map["british_museum"]["cal_df"][train_map["british_museum"]["cal_df"].image_id=="93658023_4980549800"]
for _property in ["camera_intrinsics", "rotation_matrix", "translation_vector"]:
    print(f"\n\n>>>>> {_property.replace('_', ' ').upper()} ARRAY <<<<<\n")
    print(arr_from_str(cal_row_from_id[_property].values[0]))

print("\n\n\nIF WE EXAMINE THE CALIBRATION DATAFRAME ROW FOR THE TOP RIGHT IMAGE (ID='77723525_5227836172') WE SEE THE FOLLOWING:")
cal_row_from_id = train_map["british_museum"]["cal_df"][train_map["british_museum"]["cal_df"].image_id=="77723525_5227836172"]
for _property in ["camera_intrinsics", "rotation_matrix", "translation_vector"]:
    print(f"\n\n>>>>> {_property.replace('_', ' ').upper()} ARRAY <<<<<\n")
    print(arr_from_str(cal_row_from_id[_property].values[0]))

    
print("\n\n\n\nEXAMPLE OF NO COVISIBILITY")
plot_two_paths(
    train_map["british_museum"]["pco_df"]["image_path_1"][15395],
    train_map["british_museum"]["pco_df"]["image_path_2"][15395],
    [f"Covisibility Between Is {train_map['british_museum']['pco_df'].covisibility[15395]}",
     f"Covisibility Between Is {train_map['british_museum']['pco_df'].covisibility[15395]}"]
)

print("\n\n\nIF WE EXAMINE THE CALIBRATION DATAFRAME ROW FOR THE TOP LEFT IMAGE (ID='66757775_3535589713') WE SEE THE FOLLOWING:")
cal_row_from_id = train_map["british_museum"]["cal_df"][train_map["british_museum"]["cal_df"].image_id=="66757775_3535589713"]
for _property in ["camera_intrinsics", "rotation_matrix", "translation_vector"]:
    print(f"\n\n>>>>> {_property.replace('_', ' ').upper()} ARRAY <<<<<\n")
    print(arr_from_str(cal_row_from_id[_property].values[0]))

print("\n\n\nIF WE EXAMINE THE CALIBRATION DATAFRAME ROW FOR THE TOP RIGHT IMAGE (ID='66747696_4734591579') WE SEE THE FOLLOWING:")
cal_row_from_id = train_map["british_museum"]["cal_df"][train_map["british_museum"]["cal_df"].image_id=="66747696_4734591579"]
for _property in ["camera_intrinsics", "rotation_matrix", "translation_vector"]:
    print(f"\n\n>>>>> {_property.replace('_', ' ').upper()} ARRAY <<<<<\n")
    print(arr_from_str(cal_row_from_id[_property].values[0]))

### 4.6 PLOT EVERY SCENE ALONG WITH COVISIBILITY
---

In [ ]:
for _s in TRAIN_SCENES:
    print(f"\n\n\nRANDOM PLOT FOR {_s} LOCATION SCENE")
    pco_random_plot(train_map[_s]["pco_df"])

### 4.7 EXAMPLE MAPPING KEYPOINTS FROM ONE IMAGE TO ANOTHER (BASIC SIFT)
---

In [ ]:
# Step 1 - Pick An Image Pair And Display
DEMO_ROW = train_map["taj_mahal"]["pco_df"].iloc[850]
DEMO_IMG_ID_1 = DEMO_ROW.image_id_1
DEMO_IMG_ID_2 = DEMO_ROW.image_id_2
DEMO_PATH_1 = DEMO_ROW.image_path_1
DEMO_PATH_2 = DEMO_ROW.image_path_2

demo_img_1 = cv2.imread(DEMO_PATH_1)[..., ::-1]
demo_img_2 = cv2.imread(DEMO_PATH_2)[..., ::-1]

# FM= Fundamental Matrix
DEMO_F = arr_from_str(DEMO_ROW.fundamental_matrix)

plot_two_paths(DEMO_PATH_1, DEMO_PATH_2, _titles=[f"IMAGE ID 1: {DEMO_IMG_ID_1}", f"IMAGE ID 2: {DEMO_IMG_ID_2}"])

In [ ]:
# Step 2 - Use SIFT (One Approach) To Detect Keypoints In Images 

#
# The task is finding the relative geometry (rotation, translation) between the two cameras.
# You can read more about epipolar geometry here: https://en.wikipedia.org/wiki/Epipolar_geometry
#
# This problem is typically (but not always!) solved with sparse features.
# Let's try using SIFT, a seminal work in computer vision (https://en.wikipedia.org/wiki/Scale-invariant_feature_transform).
# No longer the state of the art, but still pretty solid!
#
n_feat = 1_000

# About parameter `contrastThreshold`:
#     - The contrast threshold used to filter out weak features in semi-uniform (low-contrast) regions. 
#     - The larger the threshold, the less features are produced by the detector.
contrast_thresh = -10_000

# About parameter `edgeThreshold`:
#     - The threshold used to filter out edge-like features. 
#     - Note that the its meaning is different from the contrastThreshold
#          --> i.e. the larger the edgeThreshold, the less features are filtered out (more features are retained)
edge_thresh = -10_000

# You may want to lower the detection threshold, as small images may not be able to reach the budget otherwise.
# Note that you may actually get more than num_features features, as a feature for one point can have multiple orientations (this is rare).
sift_detector = cv2.SIFT_create(n_feat, contrastThreshold=contrast_thresh, edgeThreshold=edge_thresh)

# Leverage sift and capture desired number of keypoints and descriptors
#  KEYPOINTS:
#     --> The keypoint is characterized by the 2D position, scale (proportional to the 
#         diameter of the neighborhood that needs to be taken into account), 
#         orientation and some other parameters. The keypoint neighborhood is then analyzed 
#         by another algorithm that builds a descriptor (usually represented as a feature vector). 
#           - the `.pt` attribute yields a tuple indicating the position of the keypoint 
#             as measured in pixels from the top-left corner of the image (x,y) (see drawn circle center)
#           - the `.angle` attribute gives the keypoint orientation (see drawn flag line)
#           - the `.size` attribute gives the keypoint magnitude/diameter (see drawn circle size)
#           - the `.response` attribute gives the keypoint detector response on the keypoint (strength of the keypoint)
#           - the `.octave` attribute gives the pyramid octave in which the keypoint has been detected
#
#  DESCRIPTORS:
#     --> A SIFT descriptor is a 3-D spatial histogram of the image gradients in 
#         characterizing the appearance of a keypoint. The gradient at each pixel 
#         is regarded as a sample of a three-dimensional elementary feature vector, 
#         formed by the pixel location and the gradient orientation. Samples are weighed 
#         by the gradient norm and accumulated in a 3-D histogram h, which 
#         (up to normalization and clamping) forms the SIFT descriptor of the region. 
#         An additional Gaussian weighting function is applied to give less importance to 
#         gradients farther away from the keypoint center. Orientations are quantized 
#         into eight bins and the spatial coordinates into four each, as follows.
#    --> A SIFT descriptor is essentially a certain number of features describing a keypoint.
#        In our case, the number of features is 8*4*4 or 128 features.
keypoints_1, descriptors_1 = extract_sift_features(demo_img_1, sift_detector, n_feat)
keypoints_2, descriptors_2 = extract_sift_features(demo_img_2, sift_detector, n_feat)

# Each local feature contains a keypoint (xy, possibly scale, possibly orientation) and a description vector (128-dimensional for SIFT).
#   DRAW_RICH_KEYPOINTS Description: 
#        --> For each keypoint the circle around keypoint with keypoint size and orientation will be drawn.
image_with_keypoints_1 = cv2.drawKeypoints(demo_img_1, keypoints_1, outImage=None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
plt.figure(figsize=(20, 15))
plt.imshow(image_with_keypoints_1)
plt.axis(False)
plt.show()

image_with_keypoints_2 = cv2.drawKeypoints(demo_img_2, keypoints_2, outImage=None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
plt.figure(figsize=(20, 15))
plt.imshow(image_with_keypoints_2)
plt.axis(False)
plt.show()

In [ ]:
# Step 3 - Match The Keypoints From Image 1 to Image 2 (Many Possible Approaches)

# For each descriptor on one image, find the closest descriptor on the other image.
# We will leverage a Brute-force descriptor matcher.
# NOTE:
#     --> With crossCheck=True we keep only bidirectional matches 
#         i.e. two features are nearest neighbours from A to B and also from B to A
bfm = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)

# Compute matches and reduce to [query|train] indexors
cv_matches = bfm.match(descriptors_1, descriptors_2)
matches_before = np.array([[m.queryIdx, m.trainIdx] for m in cv_matches])

# Convert keypoints and matches to something more human-readable.
kp_1_pos = np.array([kp.pt for kp in keypoints_1])
kp_2_pos = np.array([kp.pt for kp in keypoints_2])
kp_matches = np.array([[m.queryIdx, m.trainIdx] for m in cv_matches])

# Plot the brute-force matches.
# Notice that this includes many outliers. We can filter them with a state-of-the-art RANSAC algorithm. References:
# * https://docs.opencv.org/4.5.4/d9/d0c/group__calib3d.html#ga59b0d57f46f8677fb5904294a23d404a
# * https://opencv.org/evaluating-opencvs-new-ransacs
img_matches_before = draw_cv_matches(demo_img_1, demo_img_2, kp_1_pos, kp_2_pos, matches_before)
plt.figure(figsize=(20, 15))
plt.title('Matches BEFORE RANSAC', fontweight="bold")
plt.imshow(img_matches_before)
plt.axis(False)
plt.show()


# OpenCV gives us the Fundamental matrix after RANSAC, and a mask over the input matches. 
# The solution is clearly much cleaner, even though it may still contain outliers.

# This `F` is the prediction you'll submit to the contest.

F, inlier_mask = cv2.findFundamentalMat(kp_1_pos[matches_before[:, 0]], kp_2_pos[matches_before[:, 1]], 
                                        cv2.USAC_MAGSAC, ransacReprojThreshold=0.25, 
                                        confidence=0.99999, maxIters=10000)


matches_after = np.array([match for match, is_inlier in zip(matches_before, inlier_mask) if is_inlier])
img_matches_after = draw_cv_matches(demo_img_1, demo_img_2, kp_1_pos, kp_2_pos, matches_after)
plt.figure(figsize=(20, 15))
plt.title('Matches AFTER RANSAC', fontweight="bold")
plt.imshow(img_matches_after)
plt.axis(False)
plt.show()

# CHECK THE FIRST KEYPOINT
match_query_kp_position_1 = keypoints_1[matches_after[0][0]].pt
match_query_kp_position_1 = (int(round(match_query_kp_position_1[0])), 
                             int(round(match_query_kp_position_1[1])))
match_train_kp_position_1 = keypoints_2[matches_after[0][1]].pt
match_train_kp_position_1 = (int(round(match_train_kp_position_1[0])), 
                             int(round(match_train_kp_position_1[1])))
_tmp_1 = cv2.circle(demo_img_1.copy(), match_query_kp_position_1, 10, (255,0,0), -1)
_tmp_2 = cv2.circle(demo_img_2.copy(), match_train_kp_position_1, 10, (255,0,0), -1)

plt.figure(figsize=(20,10))
plt.subplot(1,2,1)
plt.title("First Keypoint In Image 1 (Query) AFTER RANSAC", fontweight="bold")
plt.axis(False)
plt.imshow(_tmp_1)

plt.subplot(1,2,2)
plt.title("First Keypoint In Image 2 (Train) AFTER RANSAC", fontweight="bold")
plt.axis(False)
plt.imshow(_tmp_2)

plt.tight_layout()
plt.show()

# CHECK THE SECOND KEYPOINT
match_query_kp_position_2 = keypoints_1[matches_after[1][0]].pt
match_query_kp_position_2 = (int(round(match_query_kp_position_2[0])), 
                             int(round(match_query_kp_position_2[1])))
match_train_kp_position_2 = keypoints_2[matches_after[1][1]].pt
match_train_kp_position_2 = (int(round(match_train_kp_position_2[0])), 
                             int(round(match_train_kp_position_2[1])))

_tmp_1 = cv2.circle(_tmp_1, match_query_kp_position_2, 10, (0,255,0), -1)
_tmp_2 = cv2.circle(_tmp_2, match_train_kp_position_2, 10, (0,255,0), -1)

plt.figure(figsize=(20,10))
plt.subplot(1,2,1)
plt.title("Second Keypoint In Image 1 (Query) AFTER RANSAC", fontweight="bold")
plt.axis(False)
plt.imshow(_tmp_1)

plt.subplot(1,2,2)
plt.title("Second Keypoint In Image 2 (Train) AFTER RANSAC", fontweight="bold")
plt.axis(False)
plt.imshow(_tmp_2)
plt.tight_layout()
plt.show()

# CHECK THE THIRD KEYPOINT
match_query_kp_position_3 = keypoints_1[matches_after[2][0]].pt
match_query_kp_position_3 = (int(round(match_query_kp_position_3[0])), 
                             int(round(match_query_kp_position_3[1])))
match_train_kp_position_3 = keypoints_2[matches_after[2][1]].pt
match_train_kp_position_3 = (int(round(match_train_kp_position_3[0])), 
                             int(round(match_train_kp_position_3[1])))

_tmp_1 = cv2.circle(_tmp_1, match_query_kp_position_3, 10, (0,0,255), -1)
_tmp_2 = cv2.circle(_tmp_2, match_train_kp_position_3, 10, (0,0,255), -1)

plt.figure(figsize=(20,10))
plt.subplot(1,2,1)
plt.title("Third Keypoint In Image 1 (Query) AFTER RANSAC", fontweight="bold")
plt.axis(False)
plt.imshow(_tmp_1)

plt.subplot(1,2,2)
plt.title("Third Keypoint In Image 2 (Train) AFTER RANSAC", fontweight="bold")
plt.axis(False)
plt.imshow(_tmp_2)
plt.tight_layout()
plt.show()

In [ ]:
# Step 4 - See How We Did #

# One important caveat: the scenes were reconstructed from unstructured image collections using 
# Structure-from-Motion (http://colmap.github.io), and are not up to "real-world" scale (i.e. meters, or inches).
#
# The hosts have computed a scaling factor per scene to correct this. 
# This is necessary to compute the metric correctly.
DEMO_SCALE_FACTOR = scale_map["taj_mahal"]
print(f"The Taj Mahal Scaling Factor is: {DEMO_SCALE_FACTOR}")

inlier_kp_1 = np.array([kp.pt for i, kp in enumerate(keypoints_1) if i in matches_after[:, 0]])
inlier_kp_2 = np.array([kp.pt for i, kp in enumerate(keypoints_2) if i in matches_after[:, 1]])

E, R, T = compute_essential_matrix(F, id2cal_map[DEMO_IMG_ID_1]["camera_intrinsics"], id2cal_map[DEMO_IMG_ID_2]["camera_intrinsics"], inlier_kp_1, inlier_kp_2)
q = quaternion_from_matrix(R)
T = T.flatten()

# Get the ground truth relative pose difference for this pair of images.
R1_gt = id2cal_map[DEMO_IMG_ID_1]["rotation_matrix"]
T1_gt = id2cal_map[DEMO_IMG_ID_1]["translation_vector"].reshape((3, 1))

R2_gt = id2cal_map[DEMO_IMG_ID_2]["rotation_matrix"]
T2_gt = id2cal_map[DEMO_IMG_ID_2]["translation_vector"].reshape((3, 1))

dR_gt = np.dot(R2_gt, R1_gt.T)
dT_gt = (T2_gt - np.dot(dR_gt, T1_gt)).flatten()

q_gt = quaternion_from_matrix(dR_gt)
q_gt = q_gt / (np.linalg.norm(q_gt) + 1e-15)

# Given ground truth and prediction, compute the error for the example above.
err_q, err_t = compute_error_for_example(q_gt, dT_gt, q, T, DEMO_SCALE_FACTOR)

print(f'rotation_error={err_q:.02f} (deg), translation_error={err_t:.02f} (m)', flush=True)

## 5.MODEL BASELINE
---


### 5.1 BASELINE USING SIMPLE
---

Let's create a wrapper function that can take an image pair and will return an approximate F matrix so we can submit using the basic techniques illustrated by the hosts in this [notebook ](https://www.kaggle.com/code/eduardtrulls/imc2022-training-data?rvi=1)

In [ ]:
INFERENCE_N_FEATURES      = 5_000
INFERENCE_CONTRAST_THRESH = -10_000
INFERENCE_EDGE_THRESH     = -10_000
INFERENCE_N_OCTAVES       = 3
DETECTOR_STYLE            = "SIFT" # ["SIFT"|"ORB"]
MATCHER_STYLE             = "BFM"

# IF USING ORB WE ALSO USE BEBLID
#
# BEBLID (Boosted Efficient Binary Local Image Descriptor): 
#           - A new descriptor introduced in 2020 that has been shown to improve ORB 
#             in several tasks. Since BEBLID works for several detection methods, 
#             you have to set the scale for the ORB keypoints which is 0.75~1. 
if DETECTOR_STYLE=="SIFT":
    INFERENCE_DETECTOR = cv2.SIFT_create(INFERENCE_N_FEATURES, INFERENCE_N_OCTAVES, INFERENCE_CONTRAST_THRESH, INFERENCE_EDGE_THRESH)
    INFERENCE_DESCRIPTOR = None
elif DETECTOR_STYLE=="ORB":
    INFERENCE_DETECTOR = cv2.ORB_create(INFERENCE_N_FEATURES, edgeThreshold=INFERENCE_EDGE_THRESH)
    INFERENCE_DESCRIPTOR = cv2.xfeatures2d.BEBLID_create(0.75)
else:        
    raise NotImplementedError("Only SIFT and ORB are Implemented So Far")
    
if MATCHER_STYLE=="BFM":
    INFERENCE_MATCHER = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
else:
    raise NotImplementedError("Only BruteForce Is Implemented So Far")

# We know there are ~10000 pairs of images so ideally we want this to run pretty quick
def get_fundamental_matrix_v1(img_path_1, img_path_2, detector, matcher, descriptor=None, n_feat=10_000, fancy_matching=False):
    # Step 1 - Load the Images
    img_1 = cv2.imread(img_path_1)[..., ::-1]
    img_2 = cv2.imread(img_path_2)[..., ::-1]
    
    # Step 2 - Get KPs and Descriptors (Slowest Step --> ~0.6-0.8 seconds)
    if descriptor is None:
        keypoints_1, descriptors_1 = extract_sift_features(img_1, detector, n_feat)
        keypoints_2, descriptors_2 = extract_sift_features(img_2, detector, n_feat)
    else:
        keypoints_1, descriptors_1 = descriptor.compute(img_1, extract_keypoints(img_1, detector))
        keypoints_1, descriptors_1 = keypoints_1[:n_feat], descriptors_1[:n_feat]
        
        keypoints_2, descriptors_2 = descriptor.compute(img_2, extract_keypoints(img_2, detector))
        keypoints_2, descriptors_2 = keypoints_2[:n_feat], descriptors_2[:n_feat]
    
    # Step 3 - Get position information from Keypoint objects
    kp_1_pos = np.array([kp.pt for kp in keypoints_1])
    kp_2_pos = np.array([kp.pt for kp in keypoints_2])
    
    # Step 4 - Compute Matches (Before RANSAC)(2nd Slowest Step --> ~0.1-0.2 seconds)
    
    if not fancy_matching:
        matches = matcher.match(descriptors_1, descriptors_2)
        matches = np.array([[m.queryIdx, m.trainIdx] for m in matches])
    else:
        matcher = cv2.DescriptorMatcher_create(cv2.DescriptorMatcher_BRUTEFORCE_HAMMING)
        nn_matches = matcher.knnMatch(descriptors_1, descriptors_2, 2)
        matches = np.array([[m.queryIdx, m.trainIdx] for m,n in nn_matches if m.distance<(0.8*n.distance)])
        
    # Step 5 - Use CV2 To Perform RANSAC and Get the Fundamental Matrix
    F, inlier_mask = cv2.findFundamentalMat(kp_1_pos[matches[:, 0]], kp_2_pos[matches[:, 1]], 
                                            cv2.USAC_MAGSAC, ransacReprojThreshold=0.25, 
                                            confidence=0.99999, maxIters=15000)
    
    # Step 6 - Convert our array to a flattened string 
    return " ".join([str(x) for x in F.flatten()])

### 5.2 DELF BASELINE
---

The DELF module takes an image as input and will describe noteworthy points with vectors.

In [ ]:
# Define the DELF constants... shown raw below
DELF_SCALES = tf.constant([0.25, 0.3536, 0.5, 0.7071, 1.0, 1.4142, 2.0])
DELF_SCORE_THRESH = tf.constant(100.0)
DELF_MAX_FEATURES = tf.constant(2500)

# Prepare an image tensor.
tf_demo_img_1 = tf.cast(demo_img_1, tf.float32)
tf_demo_img_2 = tf.cast(demo_img_2, tf.float32)

# Instantiate the DELF module using the offline directory
LOCAL_TFHUB_DELF_PATH = "/kaggle/input/offline-delf-from-tfhub/delf_cache/333559635dbfa88957ed2e7ed45bdbfe3353e356"
delf_module = tfhub.load(LOCAL_TFHUB_DELF_PATH).signatures['default']

delf_inputs_1 = {
  # An image tensor with dtype float32 and shape [height, width, 3], where
  # height and width are positive integers:
  'image': tf_demo_img_1,
    
  # Scaling factors for building the image pyramid as described in the paper:
  'image_scales': tf.constant([0.25, 0.3536, 0.5, 0.7071, 1.0, 1.4142, 2.0]),
    
  # Image features whose attention score exceeds this threshold will be returned:
  'score_threshold': tf.constant(100.0),
    
  # The maximum number of features that should be returned:
  'max_feature_num': tf.constant(1000),
}

# Apply the DELF module to the inputs to get the outputs.
#
# NOTE: 
#  delf_outputs is a dictionary of named tensors:
#      * delf_outputs['locations']: 
#           - a Tensor with dtype float32 and shape [None, 2],
#           - where each entry is a coordinate (vertical-offset, horizontal-offset) 
#             in pixels from the top-left corner of the image.
#      * delf_outputs['descriptors']: 
#.          - a Tensor with dtype float32 and shape [None, 40], 
#           - where delf_outputs['descriptors'][i] is a 40-dimensional
#             descriptor for the image at location delf_outputs['locations'][i]
delf_outputs_1 = delf_module(**delf_inputs_1)

delf_inputs_2 = {"image":tf_demo_img_2, 
                 "image_scales":DELF_SCALES, 
                 "score_threshold":DELF_SCORE_THRESH,
                 "max_feature_num":DELF_MAX_FEATURES}
delf_outputs_2 = delf_module(**delf_inputs_2)

In [ ]:
def delf_inference(img, delf_model,
                   _scales=(0.25, 0.3536, 0.5, 0.7071, 1.0, 1.4142, 2.0),
                   _score_thresh=100.0, _n_features=2500,):
    tf_img = tf.cast(img, tf.float32)
    input_map = {"image":tf_img, 
                 "image_scales":tf.constant(_scales), 
                 "score_threshold":tf.constant(_score_thresh),
                 "max_feature_num":tf.constant(_n_features)}
    outputs = delf_model(**input_map)
    keypoints, descriptors = outputs["locations"].numpy(), outputs["descriptors"].numpy()    
    keypoints[:, [0, 1]] = keypoints[:, [1, 0]]
    return keypoints, descriptors


def use_nn_to_filter_kps(keypoints_1, keypoints_2, descriptors_1, descriptors_2, d_thresh=0.8):
    n_kp_1, n_kp_2 = keypoints_1.shape[0], keypoints_2.shape[0]
    
    # Find nearest-neighbor matches using a KD tree.
    d1_tree = cKDTree(descriptors_1)
    _, idxs = d1_tree.query(descriptors_2, distance_upper_bound=d_thresh)

    # Select feature locations for putative matches.
    keypoints_2 = np.array([keypoints_2[i,] for i in range(n_kp_2) if idxs[i]!=n_kp_1])
    keypoints_1 = np.array([keypoints_1[idxs[i],] for i in range(n_kp_2) if idxs[i]!=n_kp_1])
    
    matches = np.array([(i,i) for i in range(keypoints_1.shape[0])])
    return keypoints_1, keypoints_2, matches

def get_fundamental_matrix_v2(img_path_1, img_path_2, detector, matcher="nn", n_feat=10_000, do_plots=False):
    # Step 1 - Load the Images
    img_1 = cv2.imread(img_path_1)[..., ::-1]/255.
    img_2 = cv2.imread(img_path_2)[..., ::-1]/255.
    
    # Step 2 - Get KPs and Descriptors
    kp_1_pos, descriptors_1 = delf_inference(img_1, detector, _n_features=n_feat)
    kp_2_pos, descriptors_2 = delf_inference(img_2, detector, _n_features=n_feat)
        
    # Step 3 - Compute Matches (Before RANSAC)(2nd Slowest Step --> ~0.1-0.2 seconds)
    if matcher=="nn":
        kp_1_pos, kp_2_pos, matches = use_nn_to_filter_kps(kp_1_pos, kp_2_pos, 
                                                           descriptors_1, descriptors_2)
    else:
        matches = matcher.match(descriptors_1, descriptors_2)
        matches = np.array([[m.queryIdx, m.trainIdx] for m in matches])
        
    if do_plots:
        img_matches_before = draw_cv_matches(255*img_1.copy(), 255*img_2.copy(), kp_1_pos, kp_2_pos, matches)
        plt.figure(figsize=(20, 15))
        plt.title('Matches BEFORE RANSAC', fontweight="bold")
        plt.imshow(img_matches_before)
        plt.axis(False)
        plt.show()
        
    # Step 4 - Use CV2 To Perform RANSAC and Get the Fundamental Matrix
    F, inlier_mask = cv2.findFundamentalMat(kp_1_pos[matches[:, 0]], kp_2_pos[matches[:, 1]], 
                                            cv2.USAC_MAGSAC, ransacReprojThreshold=0.25, 
                                            confidence=0.99999, maxIters=100_000)
    
    if do_plots:
        matches = np.array([match for match, is_inlier in zip(matches, inlier_mask) if is_inlier])
        img_matches_after = draw_cv_matches(255*img_1.copy(), 255*img_2.copy(), kp_1_pos, kp_2_pos, matches)
        plt.figure(figsize=(20, 15))
        plt.title('Matches AFTER RANSAC', fontweight="bold")
        plt.imshow(img_matches_after)
        plt.axis(False)
        plt.show()
    
    # Step 6 - Convert our array to a flattened string 
    return " ".join([str(x) for x in F.flatten()])

print("\n\nDEMO WITH REGULAR MATCHER\n")
DEMO_F = get_fundamental_matrix_v2(DEMO_PATH_1, DEMO_PATH_2, detector=delf_module, matcher=INFERENCE_MATCHER, n_feat=500, do_plots=True)

print("\n\nDEMO WITH KNN MATCHER\n")
DEMO_F = get_fundamental_matrix_v2(DEMO_PATH_1, DEMO_PATH_2, detector=delf_module, matcher="nn", n_feat=500, do_plots=True)

## 6. SUBMISSION
---

**For this submission I will use the KNN matcher and the newly minted DELF module**

In [ ]:
all_test_image_paths_1 = test_df.f_path_1.tolist()
all_test_image_paths_2 = test_df.f_path_2.tolist()

if len(all_test_image_paths_1)==3:
    ss_df["fundamental_matrix"] = [get_fundamental_matrix_v2(_path_1, _path_2, detector=delf_module, matcher="nn", do_plots=True) for _path_1, _path_2 in zip(all_test_image_paths_1, all_test_image_paths_2)]
else:
    ss_df["fundamental_matrix"] = [get_fundamental_matrix_v2(_path_1, _path_2, detector=delf_module, matcher=INFERENCE_MATCHER) for _path_1, _path_2 in zip(all_test_image_paths_1, all_test_image_paths_2)]
    
display(ss_df)
ss_df.to_csv("submission.csv", index=False)

**Thank you!**